In [1]:
import numpy as np
from keras import models
from keras import layers
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification

In [3]:
# 랜덤 시드 설정
np.random.seed(0)

# 특성 개수
number_of_features = 100

# 특성 행렬과 타깃 벡터
features, target = make_classification(n_samples=10000, 
                                       n_features=number_of_features,
                                       n_informative=3, n_redundant=0, n_classes=2, 
                                       weights=[.5, .5], random_state=0)

In [5]:
#신경망 모델
def create_network():
    network = models.Sequential()
    network.add(layers.Dense(units=16, activation='relu', name='hidden-1',
                             input_shape=(number_of_features, )))
    network.add(layers.Dense(units=16, activation='relu', name='hidden-2'))
    network.add(layers.Dense(units=1, activation='sigmoid', name='output'))
    network.compile(loss='binary_crossentropy', optimizer='rmsprop', 
                    metrics=['accuracy'])
    return network

In [6]:
#모델 생성
from scikeras.wrappers import KerasClassifier
neural_network = KerasClassifier(model=create_network, epochs=10, 
                                 batch_size=100, verbose=0)

In [7]:
#교차검증을 사용하여 모델 평가
cross_val_score(neural_network, features, target, cv=3)

array([0.90911818, 0.90759076, 0.89888989])

In [8]:
#하이퍼 파라미터 튜닝
from sklearn.model_selection import GridSearchCV

# 랜덤 시드 설정
np.random.seed(0)

# 특성 개수
number_of_features = 10

# 특성 행렬과 타깃 벡터
features, target = make_classification(n_samples=100, 
                                       n_features=number_of_features,
                                       n_informative=3, n_redundant=0, n_classes=2, 
                                       weights=[.5, .5], random_state=0)

In [9]:
#설정된 신경망을 반환하는 함수
def create_network(optimizer='rmsprop'):
    network = models.Sequential()
    network.add(layers.Dense(units=16, activation='relu', name='hidden-1',
                             input_shape=(number_of_features, )))
    network.add(layers.Dense(units=16, activation='relu', name='hidden-2'))
    network.add(layers.Dense(units=1, activation='sigmoid', name='output'))
    network.compile(loss='binary_crossentropy', optimizer=optimizer, 
                    metrics=['accuracy'])
    return network

In [10]:
neural_network = KerasClassifier(model=create_network, verbose=0)

# 하이퍼 파라미터 탐색영역 정의
epochs = [5, 10]
batches = [5, 10, 100]
optimizers = ['rmsprop', 'adam']

# 하이퍼파라미터 그리드
hyperparameters = dict(optimizer=optimizers, epochs=epochs, batch_size=batches)

# 그리드 서치
grid = GridSearchCV(estimator=neural_network, param_grid=hyperparameters)

# 그리드 서치 수행
grid_result = grid.fit(features, target)

In [11]:
#최적의 하이퍼파라미터 확인
grid_result.best_params_

{'batch_size': 10, 'epochs': 10, 'optimizer': 'adam'}

In [12]:
grid_result.cv_results_['mean_test_score']

array([0.66, 0.6 , 0.61, 0.62, 0.59, 0.59, 0.6 , 0.68, 0.57, 0.46, 0.56,
       0.53])

In [13]:
grid_result.best_estimator_

KerasClassifier(
	model=<function create_network at 0x00000252855E5C10>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=adam
	loss=None
	metrics=None
	batch_size=10
	validation_batch_size=None
	verbose=0
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=10
	class_weight=None
)